In [1]:
!python -m pip install --upgrade pip
!python -m pip install -r requirements.txt


  Using cached pip-25.2-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import scrapy

print("✅ All packages imported successfully!")


✅ All packages imported successfully!


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [4]:
# Define the URL and headers
url = "https://www.treasury.gov.lk"
headers = {
    'User-Agent': 'Group 5 DA2009 Project Scraper/1.0 (https://github.com/your_repo_name)'
}

# Add a User-Agent to your requests to mimic a browser.
# This is a key step in ethical scraping, as covered in Week 7[cite: 1158].

In [5]:
try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Check for bad status codes (4xx or 5xx) [cite: 518, 532]
    soup = BeautifulSoup(response.content, 'html.parser')
    print("Successfully fetched the page.")
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

Successfully fetched the page.


In [6]:
# Assuming news headlines are inside <h2> tags with a specific class
news_items = soup.find_all('h2', class_='news-title')
scraped_data = []

# Loop through each item to extract the text and a link
for item in news_items:
    title = item.get_text(strip=True)
    link_tag = item.find('a')
    if link_tag:
        link = link_tag['href']
        scraped_data.append({'Title': title, 'Link': url + link})

# Convert to a DataFrame and display
df = pd.DataFrame(scraped_data)
df

""


In [7]:
# Try to scrape any HTML tables from the page
try:
    tables = pd.read_html(url)
    print(f"Found {len(tables)} tables on the page.")
    # The first table is usually at index 0. You can inspect others if needed.
    df_table = tables[0]
    print(df_table.head())
except Exception as e:
    print(f"No tables found or an error occurred: {e}")

Found 4 tables on the page.
  Currency     Buying    Selling
0      USD  297.08LKR  304.61LKR
1      GBP  397.64LKR  410.70LKR


In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up the driver (make sure the chromedriver is in your path or folder)
driver = webdriver.Chrome()

try:
    driver.get(url)

    # Wait for up to 10 seconds for the dynamic content (e.g., a table with the ID 'data-table') to be visible [cite: 1053]
    WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.ID, 'data-table'))
    )

    # Now that the element has loaded, you can get the page source
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # Scrape the dynamic content from the soup object
    dynamic_data_element = soup.find('table', id='data-table')

    # Now, you can process the dynamic_data_element with BeautifulSoup
    # ... your scraping logic here ...

finally:
    # Always remember to close the browser
    driver.quit()

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff7bbc7e415+77285]
	GetHandleVerifier [0x0x7ff7bbc7e470+77376]
	(No symbol) [0x0x7ff7bba49a6a]
	(No symbol) [0x0x7ff7bbaa0406]
	(No symbol) [0x0x7ff7bbaa06bc]
	(No symbol) [0x0x7ff7bbaf3ac7]
	(No symbol) [0x0x7ff7bbac864f]
	(No symbol) [0x0x7ff7bbaf087f]
	(No symbol) [0x0x7ff7bbac83e3]
	(No symbol) [0x0x7ff7bba91521]
	(No symbol) [0x0x7ff7bba922b3]
	GetHandleVerifier [0x0x7ff7bbf61efd+3107021]
	GetHandleVerifier [0x0x7ff7bbf5c29d+3083373]
	GetHandleVerifier [0x0x7ff7bbf7bedd+3213485]
	GetHandleVerifier [0x0x7ff7bbc9884e+184862]
	GetHandleVerifier [0x0x7ff7bbca055f+216879]
	GetHandleVerifier [0x0x7ff7bbc87084+113236]
	GetHandleVerifier [0x0x7ff7bbc87239+113673]
	GetHandleVerifier [0x0x7ff7bbc6e298+11368]
	BaseThreadInitThunk [0x0x7ffa8df5e8d7+23]
	RtlUserThreadStart [0x0x7ffa8f69c34c+44]


In [10]:
# Save the scraped data to a CSV file
df.to_csv('data/scraped_treasury_data.csv', index=False)
print("Data saved successfully to data/scraped_treasury_data.csv")

Data saved successfully to data/scraped_treasury_data.csv


In [3]:
url = "https://www.treasury.gov.lk/"
tables = pd.read_html(url)  # returns list of DataFrames
df = tables[0]  # first table


In [4]:
import pandas as pd

url = "https://www.treasury.gov.lk/"
tables = pd.read_html(url)

print(f"Found {len(tables)} tables")

for i, table in enumerate(tables):
    print(f"\nTable {i} shape: {table.shape}")
    print(table.head())


Found 4 tables

Table 0 shape: (2, 3)
  Currency     Buying    Selling
0      USD  297.08LKR  304.61LKR
1      GBP  397.64LKR  410.70LKR

Table 1 shape: (3, 3)
                    Unnamed: 0   Year / Month Amount / LKR Bn
0  Governement Revenue & Grant  Jan-July 2024          2155.9
1       Government Expenditure  Jan-July 2024          3034.4
2       Overall Budget Deficit              .               .

Table 2 shape: (2, 4)
   #               Item   Pettah Dambulla
0  1              Samba  0LKR/Kg  0LKR/Kg
1  2  Red-Onions(Local)  0LKR/Kg  0LKR/Kg

Table 3 shape: (2, 4)
     Month/Year  Export  Import  Trade Balance
0  Jan-Nov 2023       0       0              0
1  Jan-Nov 2022       0       0              0


In [5]:
df = tables[2]  # replace with correct index
df.to_csv("treasury_data_raw.csv", index=False)


In [6]:
import os

folders = ["data_raw", "data_clean", "slides", "docs"]
for folder in folders:
    os.makedirs(folder, exist_ok=True)

print("Project folders created:", folders)


Project folders created: ['data_raw', 'data_clean', 'slides', 'docs']


In [7]:
print("Test notebook")


Test notebook


Tute 1 excersises for MOF

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Test URL
url = "https://www.treasury.gov.lk/"
response = requests.get(url)

print("✅ Status:", response.status_code)

# Try reading tables
try:
    tables = pd.read_html(url)
    print(f"Found {len(tables)} tables")
    display(tables[0].head())
except Exception as e:
    print("No HTML tables found:", e)

# Try parsing with BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")
print("✅ Page title:", soup.title.string)


✅ Status: 200
Found 4 tables


,Currency,Buying,Selling
0,USD,297.35LKR,304.87LKR
1,GBP,401.53LKR,414.18LKR


✅ Page title: Ministry of Finance - Sri lanka


Get all links on homepage

In [2]:
import requests
from bs4 import BeautifulSoup

url = "https://www.treasury.gov.lk/"
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")

# Extract all links
links = [a.get("href") for a in soup.find_all("a", href=True)]
print("Total links found:", len(links))

# Show first 10
links[:10]


Total links found: 96


['#home',
 '#budget-highlights',
 '#at-a-glance',
 '#mof-links',
 '/',
 '/si/#',
 '/ta/#',
 '/#',
 '/search',
 '/']

Useful links

In [3]:
base_url = "https://www.treasury.gov.lk"

# Keep only proper links (no '#' anchors)
valid_links = [link for link in links if link and not link.startswith("#")]

# Convert relative URLs (like "/search") into full URLs
full_links = [
    link if link.startswith("http") else base_url + link
    for link in valid_links
]

print("Valid links found:", len(full_links))
full_links[:10]


Valid links found: 92


['https://www.treasury.gov.lk/',
 'https://www.treasury.gov.lk/si/#',
 'https://www.treasury.gov.lk/ta/#',
 'https://www.treasury.gov.lk/#',
 'https://www.treasury.gov.lk/search',
 'https://www.treasury.gov.lk/',
 'https://www.treasury.gov.lk/web/about',
 'https://www.treasury.gov.lk/web/about/section/purview%20and%20function',
 'https://www.treasury.gov.lk/web/about/section/minister%20of%20finance',
 'https://www.treasury.gov.lk/web/about/section/deputy%20ministers']

Filter only PDF links

In [4]:
pdf_links = [link for link in links if link.endswith(".pdf")]
print("PDF files found:", len(pdf_links))
pdf_links[:5]


PDF files found: 0


[]

In [5]:
import requests
from bs4 import BeautifulSoup

base_url = "https://www.treasury.gov.lk"
visited = set()
pdf_links = []

# Start with homepage
to_visit = [base_url]

while to_visit:
    url = to_visit.pop()
    if url in visited:
        continue
    visited.add(url)

    try:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "html.parser")

        # Extract all links on this page
        links = [a.get("href") for a in soup.find_all("a") if a.get("href")]

        for link in links:
            # Full link
            full = link if link.startswith("http") else base_url + link

            # If PDF, save it
            if full.endswith(".pdf"):
                pdf_links.append(full)

            # If it's still inside treasury.gov.lk, add for crawling
            elif base_url in full:
                to_visit.append(full)

    except Exception as e:
        print("Error visiting:", url, e)

print("Total PDF files found:", len(pdf_links))
for pdf in pdf_links[:10]:
    print(pdf)


Error visiting: https://www.treasury.gov.lk http://www.portcitycolombo.gov.lk HTTPSConnectionPool(host='www.treasury.gov.lk%20http', port=443): Max retries exceeded with url: /www.portcitycolombo.gov.lk (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000020D21BE6090>: Failed to resolve 'www.treasury.gov.lk%20http' ([Errno 11001] getaddrinfo failed)"))


KeyboardInterrupt: 

In [6]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

base_url = "https://www.treasury.gov.lk/web/guest/home"
to_visit = [base_url]
visited = set()
pdf_links = []

while to_visit:
    url = to_visit.pop()
    if url in visited:
        continue
    visited.add(url)

    try:
        r = requests.get(url, timeout=10)
        soup = BeautifulSoup(r.text, "html.parser")

        for a in soup.find_all("a", href=True):
            link = a["href"]
            full_url = urljoin(url, link)

            if full_url.endswith(".pdf"):
                pdf_links.append(full_url)
            elif base_url in full_url and full_url not in visited:
                to_visit.append(full_url)
    except Exception as e:
        print("Skipping:", url, "Error:", e)

print("Total PDFs found:", len(pdf_links))
print(pdf_links[:10])  # show first 10


Total PDFs found: 0
[]


PDF files in budegt

In [10]:
pip install camelot-py tabula-py pdfplumber


   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.0 MB 5.6 MB/s eta 0:00:03
   ------ --------------------------------- 2.1/12.0 MB 5.6 MB/s eta 0:00:02
   --------- ------------------------------ 2.9/12.0 MB 5.2 MB/s eta 0:00:02
   ------------- -------------------------- 4.2/12.0 MB 5.1 MB/s eta 0:00:02
   ----------------- ---------------------- 5.2/12.0 MB 5.1 MB/s eta 0:00:02
   -------------------- ------------------- 6.3/12.0 MB 5.1 MB/s eta 0:00:02
   ------------------------ --------------- 7.3/12.0 MB 5.1 MB/s eta 0:00:01
   --------------------------- ------------ 8.4/12.0 MB 5.1 MB/s eta 0:00:01
   ------------------------------- -------- 9.4/12.0 MB 5.1 MB/s eta 0:00:01
   ----------------------------------- ---- 10.7/12.0 MB 5.2 MB/s eta 0:00:01
   ---------------------------------------  11.8/12.0 MB 5.1 MB/s eta 0:00:01
   ---------------------------------------- 12.0/12.0 MB 5.1 MB/s  0:00:02
   ---

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.2.6 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.


  Attempting uninstall: numpy
   ---------------------------------------- 0/6 [pypdf]
    Found existing installation: numpy 1.26.4
   ---------------------------------------- 0/6 [pypdf]
   ------ --------------------------------- 1/6 [numpy]
   ------ --------------------------------- 1/6 [numpy]
    Uninstalling numpy-1.26.4:
   ------ --------------------------------- 1/6 [numpy]
      Successfully uninstalled numpy-1.26.4
   ------ --------------------------------- 1/6 [numpy]
   ------ --------------------------------- 1/6 [numpy]
   ------ --------------------------------- 1/6 [numpy]
   ------ --------------------------------- 1/6 [numpy]
   ------ --------------------------------- 1/6 [numpy]
   ------ --------------------------------- 1/6 [numpy]
   ------ --------------------------------- 1/6 [numpy]
   ------ --------------------------------- 1/6 [numpy]
   ------ --------------------------------- 1/6 [numpy]
   ------ --------------------------------- 1/6 [numpy]
   ------

In [3]:
import pdfplumber
import pandas as pd

pdf_path = "Budget Speech 2025-rp.pdf"

tables = []
with pdfplumber.open(pdf_path) as pdf:
    for i, page in enumerate(pdf.pages, start=1):
        page_tables = page.extract_tables()
        for table in page_tables:
            df = pd.DataFrame(table)
            df["page"] = i
            tables.append(df)

print(f"✅ Extracted {len(tables)} tables from the PDF")

# Preview first table if available
if tables:
    print(tables[0].head())
else:
    print("⚠️ No tables detected")



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\DELL\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\DELL\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\DELL\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "c:\Users\DELL\anaconda3\Lib\site-packages

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import

In [1]:
import os

output_folder = "data_raw/tables"
os.makedirs(output_folder, exist_ok=True)

for i, table in enumerate(tables):
    csv_path = os.path.join(output_folder, f"table_{i+1}.csv")
    table.to_csv(csv_path, index=False)
    print(f"Saved: {csv_path}")


NameError: name 'tables' is not defined

In [4]:
pip install "numpy<2.0"


   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/15.5 MB 4.2 MB/s eta 0:00:04
   ---- ----------------------------------- 1.6/15.5 MB 4.7 MB/s eta 0:00:03
   ------ --------------------------------- 2.6/15.5 MB 5.0 MB/s eta 0:00:03
   -------- ------------------------------- 3.4/15.5 MB 4.8 MB/s eta 0:00:03
   ---------- ----------------------------- 4.2/15.5 MB 4.4 MB/s eta 0:00:03
   -------------- ------------------------- 5.5/15.5 MB 4.7 MB/s eta 0:00:03
   ---------------- ----------------------- 6.6/15.5 MB 4.7 MB/s eta 0:00:02
   ------------------ --------------------- 7.3/15.5 MB 4.6 MB/s eta 0:00:02
   --------------------- ------------------ 8.4/15.5 MB 4.7 MB/s eta 0:00:02
   ------------------------ --------------- 9.4/15.5 MB 4.7 MB/s eta 0:00:02
   --------------------------- ------------ 10.7/15.5 MB 4.8 MB/s eta 0:00:01
   ------------------------------ --------- 11.8/15.5 MB 4.9 MB/s eta 0:00:01
   -

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


In [7]:
import pdfplumber
import pandas as pd

pdf_path = "Budget Speech 2025-rp.pdf"

tables = []
with pdfplumber.open(pdf_path) as pdf:
    for i, page in enumerate(pdf.pages, start=1):
        page_tables = page.extract_tables()
        for table in page_tables:
            df = pd.DataFrame(table)
            df["page"] = i
            tables.append(df)

print(f"✅ Extracted {len(tables)} tables from the PDF")

# Preview first table if available
if tables:
    print(tables[0].head())
else:
    print("⚠️ No tables detected")



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\DELL\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\DELL\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\DELL\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "c:\Users\DELL\anaconda3\Lib\site-packages

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import

In [6]:
import numpy as np
print(np.__version__)


2.2.6


In [10]:
import pdfplumber
import pandas as pd

pdf_path = "Budget Speech 2025-rp.pdf"

tables = []
with pdfplumber.open(pdf_path) as pdf:
    for i, page in enumerate(pdf.pages, start=1):
        page_tables = page.extract_tables()
        for table in page_tables:
            df = pd.DataFrame(table)
            df["page"] = i
            tables.append(df)

print(f"✅ Extracted {len(tables)} tables from the PDF")

# Preview first table if available
if tables:
    print(tables[0].head())
else:
    print("⚠️ No tables detected")

✅ Extracted 116 tables from the PDF
      0     1     2            3     4     5       6     7  page
0  Item  2023               2024                2025         159
1  None  None  None  Provisional  None  None  Budget  None   159


In [8]:
pip install "numpy<2.0"


Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
print(np.__version__)


1.26.4


In [9]:
import os

output_folder = "data_raw/tables"
os.makedirs(output_folder, exist_ok=True)

for i, table in enumerate(tables):
    csv_path = os.path.join(output_folder, f"table_{i+1}.csv")
    table.to_csv(csv_path, index=False)
    print(f"Saved: {csv_path}")


Saved: data_raw/tables\table_1.csv
Saved: data_raw/tables\table_2.csv
Saved: data_raw/tables\table_3.csv
Saved: data_raw/tables\table_4.csv
Saved: data_raw/tables\table_5.csv
Saved: data_raw/tables\table_6.csv
Saved: data_raw/tables\table_7.csv
Saved: data_raw/tables\table_8.csv
Saved: data_raw/tables\table_9.csv
Saved: data_raw/tables\table_10.csv
Saved: data_raw/tables\table_11.csv
Saved: data_raw/tables\table_12.csv
Saved: data_raw/tables\table_13.csv
Saved: data_raw/tables\table_14.csv
Saved: data_raw/tables\table_15.csv
Saved: data_raw/tables\table_16.csv
Saved: data_raw/tables\table_17.csv
Saved: data_raw/tables\table_18.csv
Saved: data_raw/tables\table_19.csv
Saved: data_raw/tables\table_20.csv
Saved: data_raw/tables\table_21.csv
Saved: data_raw/tables\table_22.csv
Saved: data_raw/tables\table_23.csv
Saved: data_raw/tables\table_24.csv
Saved: data_raw/tables\table_25.csv
Saved: data_raw/tables\table_26.csv
Saved: data_raw/tables\table_27.csv
Saved: data_raw/tables\table_28.csv
S

In [4]:
all_tables = pd.concat(tables, ignore_index=True)
all_tables.to_csv("data_raw/all_budget_tables.csv", index=False)
print("💾 All tables combined into: data_raw/all_budget_tables.csv")


💾 All tables combined into: data_raw/all_budget_tables.csv


In [5]:
df = all_tables.dropna(how="all")  # remove empty rows
df.columns = [str(c).strip() for c in df.columns]  # clean col names


In [6]:
# Remove fully empty rows
df = all_tables.dropna(how="all")

# Remove fully empty columns
df = df.dropna(axis=1, how="all")

# Clean column names (strip spaces, unify names)
df.columns = [str(c).strip() for c in df.columns]

# Remove duplicate header rows (where "Item" or "2023" appears again mid-table)
df = df[df.columns].loc[df[df.columns[0]] != "Item"]

# Reset index
df = df.reset_index(drop=True)

# Save cleaned version
df.to_csv("data_raw/cleaned_budget_tables.csv", index=False)

print("✨ Cleaned CSV saved as: data_raw/cleaned_budget_tables.csv")
print(df.head())


✨ Cleaned CSV saved as: data_raw/cleaned_budget_tables.csv
      0                                                1     2            3  \
0  None                                             None  None  Provisional   
1                                                   Item                      
2        Total Receipts other than Government Borrowings                      
3                              Total Primary Expenditure                      
4                                              Recurrent                      

             4     5       6     7  page    8  ...   12   13   14   15   16  \
0         None  None  Budget  None   159  NaN  ...  NaN  NaN  NaN  NaN  NaN   
1  Rs. Billion           NaN   NaN   160  NaN  ...  NaN  NaN  NaN  NaN  NaN   
2        5,042           NaN   NaN   160  NaN  ...  NaN  NaN  NaN  NaN  NaN   
3        4,285           NaN   NaN   160  NaN  ...  NaN  NaN  NaN  NaN  NaN   
4        2,970           NaN   NaN   160  NaN  ...  NaN  NaN  NaN  NaN 

In [7]:
import pandas as pd
import pdfplumber

pdf_path = "Budget Speech 2025-rp.pdf"

tables = []
with pdfplumber.open(pdf_path) as pdf:
    for i, page in enumerate(pdf.pages, start=1):
        page_tables = page.extract_tables()
        for table in page_tables:
            df = pd.DataFrame(table)
            
            # Skip empty or junk tables (less than 3 columns or 3 rows)
            if df.shape[1] < 3 or df.shape[0] < 3:
                continue
            
            # Keep page info
            df["page"] = i
            tables.append(df)

print(f"✅ Extracted {len(tables)} clean tables")

# Concatenate only valid tables
if tables:
    all_tables = pd.concat(tables, ignore_index=True)
    
    # Drop rows/columns that are all NaN
    all_tables = all_tables.dropna(how="all").dropna(axis=1, how="all")
    
    # Save cleaned file
    all_tables.to_csv("data_raw/filtered_budget_tables.csv", index=False)
    print("💾 Saved as data_raw/filtered_budget_tables.csv")
    
    # Preview
    print(all_tables.head(10))
else:
    print("⚠️ No valid tables detected")


✅ Extracted 46 clean tables
💾 Saved as data_raw/filtered_budget_tables.csv
  0                                                  1 2 3            4 5  \
0                                                 Item      Rs. Billion     
1      Total Receipts other than Government Borrowings            5,042     
2                            Total Primary Expenditure            4,285     
3                                            Recurrent            2,970     
4                                              Capital            1,315     
5                                Debt Service Payments            4,550     
6                                    Interest Payments            2,950     
7                                      Debt Repayments            1,600     
8                      Provision for Advanced Accounts                7     
9    Adjustments for book/cash Value of Government ...              200     

   page    6    7    8  ...   12   13   14   15   16   17   18   19   20   21

In [11]:
import os

# Folder to save individual tables
output_folder = "data_raw/tables"
os.makedirs(output_folder, exist_ok=True)

# Save each table separately
for i, table in enumerate(tables, start=1):  # start=1 for table_1, table_2...
    try:
        # Convert to DataFrame if not already
        df = pd.DataFrame(table)
        
        # File path
        csv_path = os.path.join(output_folder, f"table_{i}.csv")
        
        # Save
        df.to_csv(csv_path, index=False)
        print(f"✅ Saved: {csv_path}")
    except Exception as e:
        print(f"⚠️ Could not save table {i}: {e}")


✅ Saved: data_raw/tables\table_1.csv
✅ Saved: data_raw/tables\table_2.csv
✅ Saved: data_raw/tables\table_3.csv
✅ Saved: data_raw/tables\table_4.csv
✅ Saved: data_raw/tables\table_5.csv
✅ Saved: data_raw/tables\table_6.csv
✅ Saved: data_raw/tables\table_7.csv
✅ Saved: data_raw/tables\table_8.csv
✅ Saved: data_raw/tables\table_9.csv
✅ Saved: data_raw/tables\table_10.csv
✅ Saved: data_raw/tables\table_11.csv
✅ Saved: data_raw/tables\table_12.csv
✅ Saved: data_raw/tables\table_13.csv
✅ Saved: data_raw/tables\table_14.csv
✅ Saved: data_raw/tables\table_15.csv
✅ Saved: data_raw/tables\table_16.csv
✅ Saved: data_raw/tables\table_17.csv
✅ Saved: data_raw/tables\table_18.csv
✅ Saved: data_raw/tables\table_19.csv
✅ Saved: data_raw/tables\table_20.csv
✅ Saved: data_raw/tables\table_21.csv
✅ Saved: data_raw/tables\table_22.csv
✅ Saved: data_raw/tables\table_23.csv
✅ Saved: data_raw/tables\table_24.csv
✅ Saved: data_raw/tables\table_25.csv
✅ Saved: data_raw/tables\table_26.csv
✅ Saved: data_raw/tab

Tute 2

In [12]:
# === Week 2: Pandas Practice ===

import pandas as pd
import os

# 1. LOAD RAW DATA
raw_file = "data_raw/tables/table_1.csv"   # pick any saved table
df = pd.read_csv(raw_file)

print("✅ File loaded successfully!")
print("📊 Shape of data:", df.shape)
display(df.head())

# 2. INSPECT DATA
print("\n--- INFO ---")
print(df.info())

print("\n--- DESCRIPTIVE STATS ---")
print(df.describe(include="all"))

# 3. CLEAN COLUMN NAMES
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
print("\n✅ Cleaned column names:", df.columns.tolist())

# 4. HANDLE MISSING VALUES
print("\n🔍 Missing values per column:")
print(df.isnull().sum())

# Example cleaning: fill missing numbers with 0, text with "Unknown"
for col in df.columns:
    if df[col].dtype in ["float64", "int64"]:
        df[col].fillna(0, inplace=True)
    else:
        df[col].fillna("Unknown", inplace=True)

# 5. REMOVE DUPLICATES
before = df.shape[0]
df.drop_duplicates(inplace=True)
after = df.shape[0]
print(f"\n🗑️ Removed {before - after} duplicate rows")

# 6. SAVE CLEANED FILE
output_file = "data_clean/table_1_clean.csv"
os.makedirs("data_clean", exist_ok=True)
df.to_csv(output_file, index=False)
print(f"\n💾 Cleaned dataset saved to: {output_file}")

# 7. PREVIEW CLEANED DATA
display(df.head())


✅ File loaded successfully!
📊 Shape of data: (2, 9)


,0,1,2,3,4,5,6,7,page
0,Item,2023.0,NaN,2024,NaN,NaN,2025,NaN,159
1,NaN,NaN,NaN,Provisional,NaN,NaN,Budget,NaN,159



--- INFO ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       1 non-null      object 
 1   1       1 non-null      float64
 2   2       0 non-null      float64
 3   3       2 non-null      object 
 4   4       0 non-null      float64
 5   5       0 non-null      float64
 6   6       2 non-null      object 
 7   7       0 non-null      float64
 8   page    2 non-null      int64  
dtypes: float64(5), int64(1), object(3)
memory usage: 276.0+ bytes
None

--- DESCRIPTIVE STATS ---
           0       1    2     3    4    5     6    7   page
count      1     1.0  0.0     2  0.0  0.0     2  0.0    2.0
unique     1     NaN  NaN     2  NaN  NaN     2  NaN    NaN
top     Item     NaN  NaN  2024  NaN  NaN  2025  NaN    NaN
freq       1     NaN  NaN     1  NaN  NaN     1  NaN    NaN
mean     NaN  2023.0  NaN   NaN  NaN  NaN   NaN  NaN  159.0
std      NaN     NaN

C:\Users\DELL\AppData\Local\Temp\ipykernel_24068\1949642375.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna("Unknown", inplace=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_24068\1949642375.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, whe

,0,1,2,3,4,5,6,7,page
0,Item,2023.0,0.0,2024,0.0,0.0,2025,0.0,159
1,Unknown,0.0,0.0,Provisional,0.0,0.0,Budget,0.0,159


In [14]:
import pandas as pd

df_check = pd.read_csv("data_clean/table_1_clean.csv")
display(df_check.head(10))   # show first 10 rows nicely formatted


,0,1,2,3,4,5,6,7,page
0,Item,2023.0,0.0,2024,0.0,0.0,2025,0.0,159
1,Unknown,0.0,0.0,Provisional,0.0,0.0,Budget,0.0,159


In [15]:
import pandas as pd

df_check = pd.read_csv("data_clean/table_1_clean.csv")
display(df_check.head(10))   # show first 10 rows nicely formatted


,0,1,2,3,4,5,6,7,page
0,Item,2023.0,0.0,2024,0.0,0.0,2025,0.0,159
1,Unknown,0.0,0.0,Provisional,0.0,0.0,Budget,0.0,159


In [16]:
import pandas as pd
import requests

url = "https://www.treasury.gov.lk"
headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers)
tables = pd.read_html(response.text)

# Check how many tables were found
print(f"Total tables found: {len(tables)}")

# Print the first table
print(tables[0].head())


C:\Users\DELL\AppData\Local\Temp\ipykernel_24068\2943246927.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


Total tables found: 4
  Currency     Buying    Selling
0      USD  297.26LKR  304.81LKR
1      GBP  401.70LKR  414.38LKR


Tute 2 gemmini

In [17]:
import requests

# Define the URL for the Ministry of Finance website
url = "https://www.treasury.gov.lk"

# Send a GET request to the URL
response = requests.get(url)

# Print the HTTP status code to check if the request was successful
print(f"Status Code: {response.status_code}")

# Check if the request was successful before printing the HTML
if response.status_code == 200:
    print("\nFirst 500 characters of the HTML content:")
    print(response.text[:500])
else:
    print("\nFailed to retrieve the page.")

Status Code: 200

First 500 characters of the HTML content:
<!DOCTYPE html><html lang="en"><head><script async="" src="https://www.googletagmanager.com/gtag/js?id=G-X5XT78QC7P"></script><script>
            window.dataLayer = window.dataLayer || [];
            function gtag(){dataLayer.push(arguments);}
            gtag('js', new Date());
            gtag('config', 'G-X5XT78QC7P', {
              page_path: window.location.pathname,
            });
          </script><meta name="viewport" content="width=device-width"/><meta charSet="utf-8"/><link href="


In [18]:
import pandas as pd
import requests

url = "https://www.treasury.gov.lk" # Replace with a URL that has a table

try:
    # Use pandas to find and read all tables on the page
    tables = pd.read_html(url)
    
    # Print the number of tables found
    print(f"Found {len(tables)} tables on the page.")
    
    # The first table is usually at index 0
    if tables:
        df_table = tables[0]
        print("\nFirst 5 rows of the first table:")
        print(df_table.head())
    
except Exception as e:
    print(f"An error occurred while reading HTML tables: {e}")

Found 4 tables on the page.

First 5 rows of the first table:
  Currency     Buying    Selling
0      USD  297.26LKR  304.81LKR
1      GBP  401.70LKR  414.38LKR


In [19]:
import pandas as pd

# This is a hypothetical URL to a CSV file on the website
csv_url = "https://www.treasury.gov.lk/data/some-report.csv"

try:
    df_csv = pd.read_csv(csv_url)
    print("Successfully read CSV data:")
    print(df_csv.head())
except Exception as e:
    print(f"Failed to read CSV from URL: {e}")

Failed to read CSV from URL: HTTP Error 404: Not Found


In [20]:
import pandas as pd
import requests

# This is a hypothetical API endpoint on the website
json_url = "https://www.treasury.gov.lk/api/v1/data"

try:
    response = requests.get(json_url)
    response.raise_for_status() # This will raise an error for bad responses
    data = response.json()
    
    # Convert the JSON data to a pandas DataFrame
    df_json = pd.DataFrame(data)
    print("Successfully read JSON data:")
    print(df_json.head())
except requests.exceptions.RequestException as e:
    print(f"Failed to retrieve data from the API: {e}")
except Exception as e:
    print(f"An error occurred while processing the JSON data: {e}")

Failed to retrieve data from the API: 404 Client Error: Not Found for url: https://www.treasury.gov.lk/api/v1/data


In [21]:
import pandas as pd
import requests

# This is a hypothetical URL to a page containing a link to a CSV
# Let's say we find the raw link by inspecting the page
raw_csv_url = "https://www.treasury.gov.lk/content/documents/2025/budget_report.csv"

try:
    df_csv = pd.read_csv(raw_csv_url)
    print("Successfully read CSV data from the raw URL:")
    print(df_csv.head())
except Exception as e:
    print(f"Error reading CSV: {e}")

# This applies to JSON too
raw_json_url = "https://www.treasury.gov.lk/api/v2/financial-data.json"

try:
    df_json = pd.read_json(raw_json_url)
    print("\nSuccessfully read JSON data from the raw URL:")
    print(df_json.head())
except Exception as e:
    print(f"Error reading JSON: {e}")

Error reading CSV: HTTP Error 404: Not Found
Error reading JSON: HTTP Error 404: Not Found


In [22]:
import requests
import pandas as pd

# This is a hypothetical API URL that returns nested JSON
# (e.g., a list of budget projects with nested details)
api_url = "https://www.treasury.gov.lk/api/v1/projects"

try:
    response = requests.get(api_url)
    response.raise_for_status()
    data = response.json()

    # The JSON data is likely a list of objects under a key, e.g., 'results'
    # We extract that list first.
    # We can inspect the data with `print(data)` to find the correct key.
    if 'projects' in data:
        results = data['projects']
    else:
        results = data # If the data is already a list

    # Use json_normalize to flatten the nested data into a DataFrame
    df_normalized = pd.json_normalize(results)
    
    print("Successfully read and flattened JSON data:")
    print(df_normalized.head())

except requests.exceptions.RequestException as e:
    print(f"Failed to retrieve data from the API: {e}")
except Exception as e:
    print(f"An error occurred while processing the JSON data: {e}")

Failed to retrieve data from the API: 404 Client Error: Not Found for url: https://www.treasury.gov.lk/api/v1/projects


BeautifulSoup-Gemini

Step 1: Set up the environment.

In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

Step 2: Define your URL and ethical headers.
As discussed in your lectures, using a proper User-Agent is an ethical best practice.



In [24]:
url = "https://www.treasury.gov.lk"
headers = {
    'User-Agent': 'Group 5 DA2009 Project Scraper/1.0 (Educational Purposes)'
}

Step 3: Fetch the HTML content.
This is where the 

requests library comes in

In [25]:
try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # This will raise an HTTPError for bad status codes
    soup = BeautifulSoup(response.content, 'html.parser')
    print("Successfully fetched the page.")
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")
    # You can choose to exit the program here if the request fails

Successfully fetched the page.


3. Extract and Store the Data
Now, let's write code to extract the data we identified. You'll need to use your browser's "Inspect" tool (F12) to find the correct HTML tags, classes, or IDs for these items.

A) Scraping News Headlines:

Assume that news headlines are within div tags with a class of "news-item" and a sub-element for the title and a link.

In [26]:
news_list = soup.find_all('div', class_='news-item')
scraped_data = []

if news_list:
    for item in news_list:
        title = item.find('h3', class_='news-title').get_text(strip=True)
        link_tag = item.find('a', class_='news-link')
        link = url + link_tag['href'] if link_tag and 'href' in link_tag.attrs else 'No Link Found'
        scraped_data.append({'Title': title, 'Link': link})

    df_news = pd.DataFrame(scraped_data)
    print("\nScraped News Headlines:")
    print(df_news.head())
    
    # Save the data to a CSV file
    df_news.to_csv('news_headlines.csv', index=False)
else:
    print("Could not find any news items with the specified class.")

Could not find any news items with the specified class.


B) Scraping HTML Tables:

If you find a table on the site, the 

pandas.read_html() function from your Week 2 lecture is the fastest way to get the data.



In [27]:
try:
    # This will search the entire page for any HTML tables
    tables = pd.read_html(response.text)
    
    if len(tables) > 0:
        df_table = tables[0] # Assume the first table is the one you want
        print(f"\nSuccessfully scraped an HTML table. Found {len(tables)} tables.")
        print("First 5 rows of the table:")
        print(df_table.head())
        
        # Save the table to a CSV file
        df_table.to_csv('financial_table.csv', index=False)
    else:
        print("\nNo HTML tables were found on the page.")
        
except Exception as e:
    print(f"\nAn error occurred while reading HTML tables: {e}")


Successfully scraped an HTML table. Found 4 tables.
First 5 rows of the table:
  Currency     Buying    Selling
0      USD  297.26LKR  304.81LKR
1      GBP  401.70LKR  414.38LKR


C:\Users\DELL\AppData\Local\Temp\ipykernel_24068\981104922.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


1. Scraping the Latest News and Press Releases
This code will navigate to the news section and extract the titles and download links for recent press releases. This is a common and practical use case for web scraping.

URL: https://www.treasury.gov.lk/web/press-releases/section/2024.

Data Points: For each press release, we want to extract its Date, Title, and the Download link.

In [28]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Define the URL for the Press Releases section
press_releases_url = "https://www.treasury.gov.lk/web/press-releases/section/2024"
base_url = "https://www.treasury.gov.lk"

headers = {
    'User-Agent': 'Group 5 DA2009 Project Scraper/1.0 (Educational Purposes)'
}

try:
    response = requests.get(press_releases_url, headers=headers)
    response.raise_for_status() # This will catch bad status codes
    soup = BeautifulSoup(response.content, 'html.parser')
    print("Successfully fetched the Press Releases page.")
    
    # Let's find the table that contains the press releases.
    # We use a CSS selector to target the specific table
    # The 'class' attribute is often a good way to identify elements.
    table = soup.find('table')
    
    if table:
        # Use pandas to read the HTML table directly into a DataFrame
        df_releases = pd.read_html(str(table))[0]
        
        # Now, you'll need to get the download links. They are usually in <a> tags.
        # This part requires inspecting the specific HTML structure.
        # We can find all links in the table and create a list.
        links = [base_url + a['href'] for a in table.find_all('a', href=True)]
        
        # Add a new column with the download links
        df_releases['Download Link'] = links[:len(df_releases)]
        
        print("\nScraped Press Release Data:")
        print(df_releases.head())
        
        # Save the DataFrame to a CSV file
        df_releases.to_csv('press_releases_data.csv', index=False)
        print("\nData saved to 'press_releases_data.csv'.")
    else:
        print("No tables found on the page.")
        
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

Successfully fetched the Press Releases page.

Scraped Press Release Data:
         Date                                              Title  Download  \
0  18-12-2024  Establishment of the Public Debt Management Of...  Download   
1  18-12-2024  Removal of Temporary Suspension on Importation...  Download   
2  18-12-2024  Sri Lanka Announces Final Results of the Invit...  Download   
3  13-12-2024  SRI LANKA ANNOUNCES THE SUCCESSFUL EXPIRATION ...  Download   
4  05-12-2024                       Supplementary Estimates-2024  Download   

                                       Download Link  
0  https://www.treasury.gov.lk/api/file/1f4a438a-...  
1  https://www.treasury.gov.lk/api/file/413ccb47-...  
2  https://www.treasury.gov.lk/api/file/8717eef6-...  
3  https://www.treasury.gov.lk/api/file/a96d4a52-...  
4  https://www.treasury.gov.lk/api/file/e5013dd3-...  

Data saved to 'press_releases_data.csv'.


C:\Users\DELL\AppData\Local\Temp\ipykernel_24068\985024911.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_releases = pd.read_html(str(table))[0]


2. Scraping Publication Links
The Ministry of Finance website also has a section for publications. We can scrape this page to get a list of all official reports and their links.

URL: https://www.treasury.gov.lk/web/publications.

Data Points: We want the titles of the publications and their corresponding links

In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Define the URL for the Publications section
publications_url = "https://www.treasury.gov.lk/web/publications"
base_url = "https://www.treasury.gov.lk"

headers = {
    'User-Agent': 'Group 5 DA2009 Project Scraper/1.0 (Educational Purposes)'
}

try:
    response = requests.get(publications_url, headers=headers)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')
    print("Successfully fetched the Publications page.")
    
    # Find all links to publications. This requires inspecting the page.
    # Publications are often in a list or a series of divs.
    # Let's assume they are within a parent div with a class 'publications-list'.
    
    publications_container = soup.find('div', class_='publications-list')
    scraped_publications = []
    
    if publications_container:
        # Find all the links within that container
        publication_links = publications_container.find_all('a', href=True)
        
        for link in publication_links:
            title = link.get_text(strip=True)
            link_href = link['href']
            
            # The link might be relative, so we join it with the base URL
            full_link = requests.compat.urljoin(base_url, link_href)
            
            scraped_publications.append({'Publication Title': title, 'Link': full_link})

        df_publications = pd.DataFrame(scraped_publications)
        print("\nScraped Publications Data:")
        print(df_publications.head())
        
        df_publications.to_csv('publications_links.csv', index=False)
        print("\nData saved to 'publications_links.csv'.")
    else:
        print("Could not find publications container on the page. Check HTML structure.")

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

Successfully fetched the Publications page.
Could not find publications container on the page. Check HTML structure.


In [30]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

url = "https://www.treasury.gov.lk/web/exchangerates/section/2024" # Example URL
headers = {'User-Agent': 'Group 5 DA2009 Project Scraper/1.0 (Educational Purposes)'}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    tables = pd.read_html(response.text)
    
    if len(tables) > 0:
        print(f"Successfully scraped an HTML page and found {len(tables)} tables.")
        
        # Access and save the first table (e.g., Currency Exchange Rates)
        df_currency = tables[0]
        df_currency.to_csv('currency_exchange_rates.csv', index=False)
        print("First table (Currency Exchange Rates) saved to CSV.")
        
        # Access and save the second table
        df_table_2 = tables[1]
        df_table_2.to_csv('table_2.csv', index=False)
        print("Second table saved to CSV.")
        
        # Access and save the third table
        df_table_3 = tables[2]
        df_table_3.to_csv('table_3.csv', index=False)
        print("Third table saved to CSV.")
        
        # Access and save the fourth table
        df_table_4 = tables[3]
        df_table_4.to_csv('table_4.csv', index=False)
        print("Fourth table saved to CSV.")
        
    else:
        print("No tables found on the page.")
        
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: 404 Client Error: Not Found for url: https://www.treasury.gov.lk/web//section/2024


In [31]:
import pandas as pd
import requests

url = "https://www.treasury.gov.lk/web/exchangerates/section/2024" # This URL may change. Please verify.
headers = {'User-Agent': 'Group 5 DA2009 Project Scraper/1.0 (Educational Purposes)'}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    tables = pd.read_html(response.text)

    if len(tables) > 0:
        print(f"Successfully scraped an HTML page and found {len(tables)} tables.")
        
        # Use a for loop to iterate through each table and save it
        for i, table in enumerate(tables):
            table_name = f'table_{i+1}.csv'
            table.to_csv(table_name, index=False)
            print(f"Table {i+1} saved to '{table_name}'.")

    else:
        print("No tables found on the page.")

except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: 404 Client Error: Not Found for url: https://www.treasury.gov.lk/web//section/2024


In [32]:
import pandas as pd
import requests

url = "https://www.treasury.gov.lk"
headers = {'User-Agent': 'Group 5 DA2009 Project Scraper/1.0 (Educational Purposes)'}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    tables = pd.read_html(response.text)

    if len(tables) > 0:
        print(f"Successfully scraped the homepage and found {len(tables)} tables.")
        
        # Save each table to a uniquely named CSV file
        for i, table in enumerate(tables):
            filename = f"table_{i+1}_economic_indicators.csv"
            table.to_csv(filename, index=False)
            print(f"Table {i+1} saved to '{filename}'.")
            
    else:
        print("No tables found on the homepage.")

except Exception as e:
    print(f"An error occurred: {e}")

Successfully scraped the homepage and found 4 tables.
Table 1 saved to 'table_1_economic_indicators.csv'.
Table 2 saved to 'table_2_economic_indicators.csv'.
Table 3 saved to 'table_3_economic_indicators.csv'.
Table 4 saved to 'table_4_economic_indicators.csv'.


C:\Users\DELL\AppData\Local\Temp\ipykernel_24068\741208.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


In [33]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# Define the base URL and the headers for ethical scraping
base_url = "https://www.treasury.gov.lk"
# **IMPORTANT**: Replace this with the actual URL that lists all departments
departments_page_url = f"{base_url}/web/ministry-and-departments" 

headers = {
    'User-Agent': 'Group 5 DA2009 Project Scraper/1.0 (Educational Purposes)'
}

scraped_data = []

print("Starting the scraping process by finding all department links...")

try:
    # Step 1: Fetch the main Departments page
    response = requests.get(departments_page_url, headers=headers)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')

    print("Successfully fetched the main departments page. Now locating links.")

    # Step 2: Find all links with the specific department URL pattern.
    # The pattern is '/web/department-of-...'
    department_links_elements = soup.find_all('a', href=re.compile(r'/web/department-of-'))

    unique_links = list(set([requests.compat.urljoin(base_url, link['href']) for link in department_links_elements]))
    
    print(f"Found {len(unique_links)} potential department links.")
    
    # Step 3: Loop through each unique department URL
    for url in unique_links:
        print(f"\nScraping page: {url}...")
        
        # Add a delay to be an ethical scraper
        time.sleep(2)
        
        try:
            department_response = requests.get(url, headers=headers)
            department_response.raise_for_status()
            department_soup = BeautifulSoup(department_response.content, 'html.parser')
            
            # Extract the department name from a heading
            department_name = department_soup.find('h1').get_text(strip=True) if department_soup.find('h1') else "Unknown"

            # Step 4: Scrape Vision and Mission statements
            vision = "Not Found"
            mission = "Not Found"
            
            # Find the headings for Vision and Mission (e.g., <h4>)
            vision_heading = department_soup.find(['h3', 'h4'], string=re.compile(r'vision', re.I))
            mission_heading = department_soup.find(['h3', 'h4'], string=re.compile(r'mission', re.I))

            # Find the text that immediately follows the headings
            if vision_heading and vision_heading.find_next_sibling(['p', 'div']):
                vision = vision_heading.find_next_sibling(['p', 'div']).get_text(strip=True)
            
            if mission_heading and mission_heading.find_next_sibling(['p', 'div']):
                mission = mission_heading.find_next_sibling(['p', 'div']).get_text(strip=True)
            
            scraped_data.append({
                "Department": department_name,
                "Vision": vision,
                "Mission": mission,
                "URL": url
            })
            
            print(f"Successfully scraped '{department_name}'. Vision: {vision != 'Not Found'}, Mission: {mission != 'Not Found'}")

        except requests.exceptions.RequestException as e:
            print(f"Error scraping {url}: {e}")
        except Exception as e:
            print(f"An error occurred while parsing {url}: {e}")

finally:
    # Step 5: Consolidate data into a single table
    if scraped_data:
        df = pd.DataFrame(scraped_data)
        df.to_excel('all_department_statements.xlsx', index=False)
        print("\nData for all departments successfully saved to 'all_department_statements.xlsx'.")
        print(df)
    else:
        print("\nNo data was scraped.")

Starting the scraping process by finding all department links...

No data was scraped.


HTTPError: 404 Client Error: Not Found for url: https://www.treasury.gov.lk/web

In [ ]:
B